In [1]:
import os
import numpy as np
import re
import pandas as pd

import csv
from shapely.geometry import Point, mapping
from fiona import collection

In [2]:
TIFF_DATA_FOLDER = 'H:/2020/tiff_files/tiffs_for_date_0' # Od kje se berejo datoteke
OUT_FOLDER = 'H:/2020/tiff_files/csv' # kam se zapisujejo rezultane datoteke (csv z -1, spucan csv (brez -1), shp datoteka)

os.makedirs(OUT_FOLDER, exist_ok=True)

BAND_NAMES = ['Blue', 'Green', 'Red', 'REdge705', 'REdge740', 'REdge783', 'NIR', 'REdge865', 
              # 'SWIR2', 
              # 'SWIR3'
             ]

BAND_INDICES = list(enumerate(BAND_NAMES, 1))

print(BAND_INDICES)

[(1, 'Blue'), (2, 'Green'), (3, 'Red'), (4, 'REdge705'), (5, 'REdge740'), (6, 'REdge783'), (7, 'NIR'), (8, 'REdge865')]


In [3]:
# Pomožne funkcije

name_pattern = re.compile(r'bandi_(?P<patch_ind>\d*?)_(?P<date_val>(\d|-)*?)\.tiff')

# Csv z -1
def out_csv_name(parsed):
    return os.path.join(OUT_FOLDER, f'full_csv_{parsed["patch_ind"]}_{parsed["date_val"]}.csv')
# Spucan csv
def bands_csv_name(parsed):
    return os.path.join(OUT_FOLDER, f'bands_csv_{parsed["patch_ind"]}_{parsed["date_val"]}.csv')
# Shapefile
def out_shp_file(parsed):
    return os.path.join(OUT_FOLDER, f'bands_out_{parsed["patch_ind"]}_{parsed["date_val"]}.shp')

def parse_filename(filename):
    m = name_pattern.match(filename)
    if m:
        return m.groupdict()
    return m

def prepare_command(filename, parsed, bands):
    in_name = os.path.join(TIFF_DATA_FOLDER, filename)
    out_name = out_csv_name(parsed)
    bands_string = ' '.join([f'-band {i}' for (i, _) in bands])
    return (f'python raster_to_pointASCII.py -skip 1 {bands_string} -csv {in_name} {out_name}', out_name)

In [4]:
# General schema for shapefile

BAND_TYPES = dict(zip(BAND_NAMES, ['float:9.6'] * len(BAND_NAMES)))
print(BAND_TYPES)

def serialize_row(row):
   
    point = Point(float(row['x']), float(row['y']))
    return {'properties': {'x' : float(row['x']),
                    'y' : float(row['y']),
                    **{band: float(row[band]) for band in BAND_NAMES},
                    'DATUM' : str(row['DATUM'])},
                'geometry': mapping(point)
           }


shp_schema = { 
    'geometry': 'Point', 
    'properties': { 
        'DATUM': 'str',
        'x': 'float:13.6',
        'y' : 'float:13.6',
        **BAND_TYPES
    }
}

print(shp_schema)

{'Blue': 'float:9.6', 'Green': 'float:9.6', 'Red': 'float:9.6', 'REdge705': 'float:9.6', 'REdge740': 'float:9.6', 'REdge783': 'float:9.6', 'NIR': 'float:9.6', 'REdge865': 'float:9.6'}
{'geometry': 'Point', 'properties': {'DATUM': 'str', 'x': 'float:13.6', 'y': 'float:13.6', 'Blue': 'float:9.6', 'Green': 'float:9.6', 'Red': 'float:9.6', 'REdge705': 'float:9.6', 'REdge740': 'float:9.6', 'REdge783': 'float:9.6', 'NIR': 'float:9.6', 'REdge865': 'float:9.6'}}


In [5]:
files = os.listdir(TIFF_DATA_FOLDER)
import time
for filename in files:
    parsed = parse_filename(filename)
    if parsed is None:
        print('Neveljavno ime datoteke', filename)
        continue
    cmd, out_file = prepare_command(filename, parsed, BAND_INDICES)
    print(cmd)
    !{cmd}
    
    out_df = pd.read_csv(out_file, header=None, names=['x', 'y'] + BAND_NAMES)
    out_df.insert(loc=0, column='RowN', value=np.arange(len(out_df)))
    
    out_df['DATUM'] = parsed['date_val'].replace('-', '.')
    
    # Če je katerakoli -1 jo vrzemo ven
    for band_name in BAND_NAMES:
        out_df = out_df[out_df[band_name] > -1]
        
    out_df.to_csv(bands_csv_name(parsed))
    t = time.time()
    # shapefile
    with collection(out_shp_file(parsed), 'w', 'ESRI Shapefile', schema=shp_schema) as output:
        rows = list(out_df.apply(serialize_row, axis=1))
        
        #rows = [serialize_row(row) for (_, row) in out_df.iterrows()]
        output.writerecords(rows)
    

Neveljavno ime datoteke BANDI_101_2020-01-11.tiff
Neveljavno ime datoteke BANDI_102_2020-01-11.tiff
Neveljavno ime datoteke BANDI_103_2020-01-11.tiff
Neveljavno ime datoteke BANDI_104_2020-01-11.tiff
Neveljavno ime datoteke BANDI_105_2020-01-11.tiff
Neveljavno ime datoteke BANDI_106_2020-01-11.tiff
Neveljavno ime datoteke BANDI_107_2020-01-11.tiff
Neveljavno ime datoteke BANDI_108_2020-01-11.tiff
Neveljavno ime datoteke BANDI_109_2020-01-11.tiff
Neveljavno ime datoteke BANDI_10_2020-01-11.tiff
Neveljavno ime datoteke BANDI_110_2020-01-11.tiff
Neveljavno ime datoteke BANDI_111_2020-01-11.tiff
Neveljavno ime datoteke BANDI_112_2020-01-11.tiff
Neveljavno ime datoteke BANDI_113_2020-01-11.tiff
Neveljavno ime datoteke BANDI_114_2020-01-11.tiff
Neveljavno ime datoteke BANDI_115_2020-01-11.tiff
Neveljavno ime datoteke BANDI_116_2020-01-11.tiff
Neveljavno ime datoteke BANDI_117_2020-01-11.tiff
Neveljavno ime datoteke BANDI_118_2020-01-11.tiff
Neveljavno ime datoteke BANDI_119_2020-01-11.tiff
N